In [2]:
import nltk
import numpy as np
import matplotlib.pyplot as plt
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD

In [3]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [4]:
word_lem = WordNetLemmatizer()
# in this notebook we will be writing our own "Tokenizer" sice this dataset contains 
# certain eords that need to be remooved 

In [5]:
titles = [line.rstrip() for line in open('all_book_titles.txt')]

In [6]:
stops = set(stopwords.words('english'))
# notice here this has been casted into a set because because 
# a few more stopwords need to be added furthur 

In [7]:
# great example of domain-specific words
# normally these below words would not be considered as stopwords 
# but here as we are dealing with a lot books we nedd to in clude them
stops = stops.union({
    'introduction','edition','series','application','approach'
})

In [8]:
def my_tokenizer(s):
  # down case
  s = s.lower()

  #split string into words (tokens)
  tokens = nltk.tokenize.word_tokenize(s)

  # remove short words they are probably not usefull
  tokens = [t for t in tokens if len(t) >2]
  # puting the words into their base form 

  tokens = [word_lem.lemmatize(t) for t in tokens]

  # removing the stopwords
  tokens = [t for t in tokens if t not in stops]

  # removing any digits
  tokens = [t for t in tokens if not any(c.isdigit() for c in t)]

  return tokens


In [9]:
vectorize  = CountVectorizer(binary =True , tokenizer = my_tokenizer)
# notice that here the bionary argumebnt has been set to true
# this means that instead of counting the tokens we just mark if the token is present or not 
# and said before we have a custom tokenzer function


In [10]:
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [11]:
X = vectorize.fit_transform(titles)

In [12]:
# now we need to create a index to word mapping 
## this is a pseudocoode of what we awant to do exactly 
#  index_word_map = [none] * len(vectorize.vocabulary_)
#for word , index in vectorize.vocabulary_.items():
#     index_word_map[index] = word

# but for us this is alredy been done as 

index_word_map = vectorize.get_feature_names_out()

In [13]:
# we take transpose of x to make rows = terms ; cols = documents
X = X.T
#However, we want to find a word vectors, which means we want to reduce the dimensionality of the term
#document matrix.
# for dimentionality reduction document sneed to be put in the columns


In [14]:
svd = TruncatedSVD()
z = svd.fit_transform(X)

In [15]:
!pip install plotly

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [16]:
import plotly.express as ex

In [17]:
fig = ex.scatter(x=z[:,0],y=z[:,1],text = index_word_map,size_max = 60)
# notice hw=ere that the first two columns have been taken as they are the modt important ones 
# that have been obtained after dimensionality reduction 
#----------------------------------------------------------------
#PLease READ NOTES  for further explaination 
#-----------------------------------------------------------
fig.update_traces(textposition = 'top center')
fig.show()